## Andrew Taylor
## atayl136
## Adv Applied Machine Learning
# Assignment 6

1. [30 pts] Load and preprocess the dataset of news articles to capture headline phrases and
other relevant fields. Note that there are 9 files and text processing would require large
corpora to be successful.
Build a tokenizer to convert each relevant word to an integer. Keep the vocabulary and
index data structures to display the generated text. (It's OK to use tokenizers from libraries
like transformers.) Make sure you create sliding sequences like shown in (Q2.) below.

In [1]:
import glob
import pandas as pd

# 1. Find all CSVs in our folder
csv_paths = glob.glob("newsarticles/*.csv")
print(f"Found {len(csv_paths)} CSV files:") 
for p in csv_paths:
    print("  ", p)
# Expect: 9 paths

# 2. Read them one by one, reporting row counts
desired_cols = ["headline", "pubDate", "articleID", "snippet", "keywords"]  # whatever you need
dfs = []
for fp in csv_paths:
    df = pd.read_csv(fp)
    print(f"  → {fp.split('/')[-1]}: {df.shape[0]} rows, columns: {df.columns.tolist()}")
    # add missing cols if you like, then subset:
    for col in desired_cols:
        if col not in df.columns:
            df[col] = ""
    dfs.append(df[desired_cols])

# 3. Concatenate and report totals
articles = pd.concat(dfs, ignore_index=True)
print(f"Total headlines loaded: {articles.shape[0]}")


# 4. Quick peek
print(articles.shape)
print(articles.head())


Found 9 CSV files:
   newsarticles\ArticlesApril2017.csv
   newsarticles\ArticlesApril2018.csv
   newsarticles\ArticlesFeb2017.csv
   newsarticles\ArticlesFeb2018.csv
   newsarticles\ArticlesJan2017.csv
   newsarticles\ArticlesJan2018.csv
   newsarticles\ArticlesMarch2017.csv
   newsarticles\ArticlesMarch2018.csv
   newsarticles\ArticlesMay2017.csv
  → newsarticles\ArticlesApril2017.csv: 886 rows, columns: ['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType', 'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate', 'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL']
  → newsarticles\ArticlesApril2018.csv: 1324 rows, columns: ['articleID', 'articleWordCount', 'byline', 'documentType', 'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate', 'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL']
  → newsarticles\ArticlesFeb2017.csv: 885 rows, columns: ['articleID', 'abstract', 'byline', 'documentType', 'headline', 'ke

In [2]:
# pip install contractions
import re
import unicodedata
import contractions

def clean_text(text: str) -> str:
    # 0) Unicode normalize (in case you have other weird punctuation)
    text = unicodedata.normalize("NFKC", text)
    
    # 1) Convert curly quotes to ASCII apostrophe
    text = text.replace("’", "'").replace("‘", "'")
    
    # 2) Strip possessive ’s (but NOT contractions like don’t, we’ll handle those next)
    #    This turns "NASA's mission" → "NASA mission", "Jones’s" → "Jones"
    text = re.sub(r"(\b\w+)'s\b", r"\1", text, flags=re.IGNORECASE)
    
    # 3) Expand standard contractions (it’s→it is, they’re→they are, etc.)
    text = contractions.fix(text)
    
    # 4) Lowercase and drop anything that’s not alnum or whitespace
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # 5) Collapse multiple spaces
    return re.sub(r"\s+", " ", text).strip()

# Apply and retrain your tokenizer on:
articles["clean_head"] = articles["headline"].fillna("").map(clean_text)



In [3]:
# Train a word-level tokenizer from scratch
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer
import numpy as np

# a) instantiate
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# b) set up trainer
trainer = WordLevelTrainer(
    vocab_size=20_000,            
    min_frequency=1,              # drop hapaxes
    special_tokens=["[PAD]", "[UNK]"]
)

# c) train on the cleaned headlines
tokenizer.train_from_iterator(articles["clean_head"].tolist(), trainer=trainer)

# 1. Inspect vocab ↔ ids
vocab = tokenizer.get_vocab()           # {token: id, …}
id_to_token = {i:t for t,i in vocab.items()}

print("Example:", list(vocab.items())[:10])
print("PAD id:", vocab["[PAD]"], "UNK id:", vocab["[UNK]"])

# 2. Encode each headline to a list of IDs
tokenized = articles["clean_head"].map(lambda t: tokenizer.encode(t).ids)

from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

# --- 3. Build & filter our windows as before ---
pad_id = vocab["[PAD]"]
max_len = 6   # 5-token context + 1 target

X2, y2, raw_ctx2 = [], [], []
for ids in tokenized:
    for i in range(1, len(ids)):           # start at 1 to ensure ≥2 tokens in context
        window = ids[: i+1]
        # left-pad so last real token is at end
        if len(window) < max_len:
            padded = [pad_id] * (max_len - len(window)) + window
        else:
            padded = window[-max_len:]
        target = padded[-1]
        if target == pad_id:               # drop pad-only targets
            continue
        X2.append(padded[:-1])
        y2.append(target)
        raw_ctx2.append(tuple(window))

X_tensor = torch.tensor(X2, dtype=torch.long)
y_tensor = torch.tensor(y2, dtype=torch.long)

# --- 4. Compute a sampling weight for each example: inverse of its context frequency ---
ctx_counts = Counter(raw_ctx2)
sample_weights = [1.0 / ctx_counts[c] for c in raw_ctx2]

# --- 5. Create a WeightedRandomSampler that oversamples rare contexts ---
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# --- 6. Build your DataLoader using that sampler (instead of shuffle=True) ---
dataset = TensorDataset(X_tensor, y_tensor)
loader  = DataLoader(
    dataset,
    batch_size=64,
    sampler=sampler,
    drop_last=True
)




Example: [('ando', 5345), ('dozen', 3852), ('wilson', 5063), ('mowing', 8236), ('spielberg', 9712), ('yards', 10575), ('strip', 9836), ('miserable', 2991), ('instinct', 4169), ('trivial', 10149)]
PAD id: 0 UNK id: 1


2. [20 pts] Build an LSTM to learn sequences of headlines. A regular embedding layer would
be helpful since the dataset is small. The entire sequence of the headline should be
machine-learned (with zero-padding as usual). The output layer should be in the
vocabulary size as we build a sequence-to-sequence model. The model computes the
conditional probabilities P(token | sequence of tokens). For every headline,
multiple sequences should be generated to calculate the probabilities by the NN, such as,
P(token2 | <token1>)
P(token3 | <token1,token2>)
P(token4 | <token1,token2,token3>)
P(token5 | <token1,token2,token3,token4>)
P(zero-pad | <token1,token2,token3,token4,token5>)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F


# 1) Modify our model to return logits for every time‐step
class HeadlineLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1, pad_idx=0):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm  = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc    = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        # x: (B, T) of token IDs (left-padded or right-padded)
        emb    = self.embed(x)          # → (B, T, E)
        out, _ = self.lstm(emb)         # → (B, T, H)
        logits = self.fc(out)           # → (B, T, V)
        return logits


# 2. Instantiate model, loss, optimizer
vocab_size = len(vocab) + 1      # +1 if your ids start at 1, with 0=PAD
embed_dim  = 256
hidden_dim = 512
num_layers = 3

model = HeadlineLSTM(vocab_size, embed_dim, hidden_dim, num_layers, pad_idx=pad_id)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 3. Training loop
n_epochs = 20
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 4. After creating optimizer:
from torch.optim.lr_scheduler import StepLR

scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
# This will cut the LR in half every 10 epochs.

# 5. Training loop with StepLR
# In our training loop, compute loss at every step:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

for epoch in range(1, n_epochs+1):
    model.train()
    total_loss = 0.0

    for xb, _ in loader:                # xb: (B, T)
        xb = xb.to(device)
        optimizer.zero_grad()

        logits = model(xb)               # → (B, T, V)
        B, T, V = logits.size()

        # shift logits & targets for teacher–forcing:
        # input tokens xb[:, :-1] predict targets xb[:, 1:]
        logits = logits[:, :-1, :].reshape(-1, V)    # → ((B*(T-1)), V)
        targets= xb[:, 1:].reshape(-1)               # → ((B*(T-1)),)

        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch}/{n_epochs} — avg loss: {total_loss/len(loader):.4f}")

    # step the scheduler
    scheduler.step()


# 6. After training, get next-token probabilities for your sample context:
sample_ctx = X_tensor[0:1].to(device)     # shape = (1, T), e.g. (1, 5)

model.eval()
with torch.no_grad():
    logits      = model(sample_ctx)       # → (1, T, V)
    final_logits= logits[:, -1, :]        # → (1, V): prediction after last token
    probs       = F.softmax(final_logits, dim=-1).squeeze(0).cpu()  # → (V,)

# 7. Top-5 predictions
print('\nTop 5 Words Generated')
top_probs, top_idx = probs.topk(5)       # each → (5,)
for idx, p in zip(top_idx.tolist(), top_probs.tolist()):
    print(f"{id_to_token[idx]:<10} {p:.4f}")



Epoch 1/20 — avg loss: 7.0752
Epoch 2/20 — avg loss: 5.9287
Epoch 3/20 — avg loss: 4.8252
Epoch 4/20 — avg loss: 3.8405
Epoch 5/20 — avg loss: 3.1418
Epoch 6/20 — avg loss: 2.7147
Epoch 7/20 — avg loss: 2.4618
Epoch 8/20 — avg loss: 2.2983
Epoch 9/20 — avg loss: 2.2090
Epoch 10/20 — avg loss: 2.1637
Epoch 11/20 — avg loss: 2.0652
Epoch 12/20 — avg loss: 2.0309
Epoch 13/20 — avg loss: 2.0133
Epoch 14/20 — avg loss: 1.9968
Epoch 15/20 — avg loss: 2.0083
Epoch 16/20 — avg loss: 1.9888
Epoch 17/20 — avg loss: 1.9813
Epoch 18/20 — avg loss: 1.9781
Epoch 19/20 — avg loss: 1.9831
Epoch 20/20 — avg loss: 1.9685

Top 5 Words Generated
meaning    0.1747
the        0.1537
a          0.1367
an         0.1073
his        0.0953


3. [10 pts] Train the model. Report the three most probable words that come after "How to".

In [5]:


# 1. Prepare the context
raw_ctx   = "How to"
clean_ctx = raw_ctx.lower()                   # → "how to"
ids       = tokenizer.encode(clean_ctx).ids   # e.g. [17, 42]

# 2. ***LEFT-pad*** to a fixed 5-token window
pad_id  = vocab["[PAD]"]
max_ctx = 5

if len(ids) < max_ctx:
    ctx_ids = [pad_id] * (max_ctx - len(ids)) + ids
else:
    ctx_ids = ids[-max_ctx:]

# 3. Turn into a tensor and send through the model
model.eval()
with torch.no_grad():
    x          = torch.tensor([ctx_ids], device=device)  # shape = (1,5)
    logits_seq = model(x)                                # shape = (1, T, V)
    # pick the logits after the final token
    final_logits = logits_seq[:, -1, :]                  # shape = (1, V)
    probs        = F.softmax(final_logits, dim=-1)       # shape = (1, V)
    probs        = probs.squeeze(0).cpu()                # shape = (V,)

# 4. Grab top-3 predictions
top_probs, top_idx = probs.topk(3)                      # each shape = (3,)

# 5. Map back to words
print('3 most probable words after How to')
for idx, p in zip(top_idx.tolist(), top_probs.tolist()):
    print(f"{id_to_token[idx]:<10} {p:.4f}")


3 most probable words after How to
get        0.0764
be         0.0719
talk       0.0561


4. [20 pts] Write a small function to query a sequence based on a chain of probabilities, such
as predicting the most probable word and then appending this word to predict the second
and third one, etc. In this way, the model can generate text.
Report the most probable three sequences that come after "How to".

In [6]:
# Sequency Query via Beam Search

def generate_beams(
    raw_ctx: str,
    model: nn.Module,
    tokenizer,
    id_to_token: dict,
    pad_id: int,
    max_ctx: int = 5,
    beam_width: int = 3,
    gen_len: int = 3,
    device: torch.device = torch.device("cpu")
):
    # 1) Tokenize & left-pad the initial context
    ids = tokenizer.encode(raw_ctx.lower()).ids
    if len(ids) < max_ctx:
        ctx = [pad_id] * (max_ctx - len(ids)) + ids
    else:
        ctx = ids[-max_ctx:]
    # beams: list of (generated_ids, log_prob)
    beams = [ (ctx.copy(), 0.0) ]
    
    model.eval()
    with torch.no_grad():
        for _ in range(gen_len):
            all_candidates = []
            for seq_ids, seq_logp in beams:
                # run the model on the current seq_ids
                x = torch.tensor([seq_ids], device=device)
                logits = model(x)                     # → (1, T, V)
                last_logits = logits[:, -1, :]        # → (1, V)
                logps = F.log_softmax(last_logits, dim=-1).squeeze(0)  # → (V,)

                # pick top beam_width next tokens
                top_logp, top_idx = logps.topk(beam_width)
                for logp, tok in zip(top_logp.tolist(), top_idx.tolist()):
                    new_seq = (seq_ids + [tok])[-max_ctx:]  # slide window
                    all_candidates.append((new_seq, seq_logp + logp))

            # keep only the top beam_width overall
            beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
    
    # Map to human‐readable
    results = []
    for seq_ids, logp in beams:
        # drop padding and the original context
        gen_part = seq_ids[-gen_len:]
        tokens   = [ id_to_token[i] for i in gen_part ]
        prob     = torch.exp(torch.tensor(logp))  # total sequence probability
        results.append((tokens, prob.item()))
    return results

# --- Example usage ---
top3 = generate_beams(
    raw_ctx="How to",
    model=model,
    tokenizer=tokenizer,
    id_to_token=id_to_token,
    pad_id=pad_id,
    max_ctx=5,
    beam_width=3,
    gen_len=3,
    device=device
)

print("Top 3 three word continuations for “How to”:")
for tokens, p in top3:
    print(f"  “How to {' '.join(tokens)}”  — P≈{p:.4f}")


Top 3 three word continuations for “How to”:
  “How to be mindful while”  — P≈0.0503
  “How to get on an”  — P≈0.0265
  “How to talk to your”  — P≈0.0221


5. [20 pts] Explore possibilities, add LSTM bi-direction, dropout, and other improvements, and
generate example text. The model can also use other fields, such as keywords, to fine-tune
the text generation.
Note your observations.

### Bidirectiional LSTM

In [7]:
# Cell 1 (fixed): “Bidirectional” LSTM that only predicts from the forward states

# make sure pad_id is defined from your vocab
pad_id  = vocab["[PAD]"]
pad_idx = pad_id

def top_k_logits(logits, k):
    v, ix    = torch.topk(logits, k)
    min_val  = v[:, -1].unsqueeze(1)
    return torch.where(logits < min_val,
                       torch.full_like(logits, -1e10),
                       logits)

def generate_sampling(
    raw_ctx, model, tokenizer, id_to_token, pad_id,
    max_ctx=5, gen_len=5,
    temperature=0.8, top_k=50,
    device="cpu"
):
    ids = tokenizer.encode(raw_ctx.lower()).ids
    ctx = ([pad_id]*(max_ctx-len(ids)) + ids)[-max_ctx:]
    seq = ctx.copy()

    model.eval()
    with torch.no_grad():
        for _ in range(gen_len):
            x      = torch.tensor([seq], device=device)
            logits = model(x)[:, -1, :]           # (1, V)
            logits = logits / temperature
            logits = top_k_logits(logits, top_k)
            probs  = F.softmax(logits, dim=-1)
            next_t = torch.multinomial(probs, num_samples=1).item()
            seq    = seq[1:] + [next_t]

    return [id_to_token[i] for i in seq[-gen_len:]]

class BiHeadlineLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, nlayers=1, pad_idx=0):
        super().__init__()
        self.hidden_dim = hid_dim
        self.embed      = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.lstm       = nn.LSTM(
            emb_dim, hid_dim, nlayers,
            batch_first=True,
            bidirectional=True
        )
        # note: still size hid_dim*2 coming out of LSTM
        self.fc         = nn.Linear(hid_dim, vocab_size)

    def forward(self, x):
        emb, _ = self.embed(x), None      # (B, T, E)
        out, _ = self.lstm(emb)           # (B, T, 2*H)
        # split into forward and backward halves
        fwd = out[:, :, :self.hidden_dim] # (B, T, H)
        return self.fc(fwd)               # (B, T, V)

# Instantiate
vocab_size = len(vocab) + 1      # +1 if your ids start at 1, with 0=PAD
embed_dim  = 128
hidden_dim = 128
num_layers = 1
model     = BiHeadlineLSTM(vocab_size, embed_dim, hidden_dim, num_layers, pad_idx).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
n_epochs  = 5

# Training loop (unchanged)
for epoch in range(1, n_epochs+1):
    model.train(); total_loss=0
    for xb, _ in loader:
        xb = xb.to(device)
        optimizer.zero_grad()
        logits = model(xb)                     # (B, T, V)
        B, T, V = logits.size()
        out = logits[:, :-1, :].reshape(-1, V) # teacher-forcing shift
        tgt = xb[:, 1:].reshape(-1)
        loss = criterion(out, tgt)
        loss.backward(); optimizer.step()
        total_loss += loss.item()
    print(f"[Bi-LSTM] Epoch {epoch}/{n_epochs} — loss: {total_loss/len(loader):.4f}")
    scheduler.step()

# Generate
print("\nBi-LSTM samples without temperature/top-k:")
print(generate_beams("How to", model, tokenizer, id_to_token, pad_id, max_ctx=5, beam_width=3, gen_len=5, device=device))

print("\nBi-LSTM samples with temperature/top-k:")
for _ in range(3):
    print("How to", " ".join(generate_sampling(
        "How to", model, tokenizer, id_to_token, pad_id,
        temperature=0.8, top_k=30, device=device
    )))


[Bi-LSTM] Epoch 1/5 — loss: 6.9954
[Bi-LSTM] Epoch 2/5 — loss: 5.6467
[Bi-LSTM] Epoch 3/5 — loss: 4.7565
[Bi-LSTM] Epoch 4/5 — loss: 4.1603
[Bi-LSTM] Epoch 5/5 — loss: 3.7443

Bi-LSTM samples without temperature/top-k:
[(['be', 'mindful', 'while', 'cleaning', 'the'], 0.003560449695214629), (['be', 'mindful', 'while', 'her', 'of'], 0.0014915864448994398), (['be', 'mindful', 'while', 'anthony', 'stars'], 0.001243238802999258)]

Bi-LSTM samples with temperature/top-k:
How to be mindful while cleaning the
How to get a wiretap to lead
How to be mindful while her you


### LSTM with Dropout

In [8]:
# Cell 2: Dropout LSTM

# make sure pad_id is defined from your vocab
pad_id = vocab["[PAD]"]
pad_idx = pad_id

class DropoutHeadlineLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, nlayers=1, pad_idx=0, emb_drop=0.2, lstm_drop=0.2):
        super().__init__()
        self.embed    = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.emb_drop = nn.Dropout(emb_drop)
        self.lstm     = nn.LSTM(emb_dim, hid_dim, nlayers,
                                batch_first=True,
                                dropout=lstm_drop if nlayers>1 else 0.0)
        self.fc       = nn.Linear(hid_dim, vocab_size)
    def forward(self, x):
        emb = self.emb_drop(self.embed(x))   # (B,T,E)
        out, _ = self.lstm(emb)               # (B,T,H)
        return self.fc(out)                  # (B,T,V)

# Instantiate
vocab_size = len(vocab) + 1      # +1 if your ids start at 1, with 0=PAD
embed_dim  = 256
hidden_dim = 512
num_layers = 2
model = DropoutHeadlineLSTM(vocab_size, embed_dim, hidden_dim, num_layers, pad_idx).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

n_epocs = 20

# Training loop
for epoch in range(1, n_epochs+1):
    model.train(); total_loss=0
    for xb, _ in loader:
        xb = xb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        B,T,V = logits.size()
        out = logits[:,:-1,:].reshape(-1,V)
        tgt = xb[:,1:].reshape(-1)
        loss = criterion(out, tgt)
        loss.backward(); optimizer.step()
        total_loss += loss.item()
    print(f"[Dropout]  Epoch {epoch}/{n_epochs} — loss: {total_loss/len(loader):.4f}")
    scheduler.step()

# Generate
print("\nDropout-LSTM samples:")
top3 = generate_beams("How to", model, tokenizer, id_to_token, pad_id, max_ctx=5, beam_width=3, gen_len=5, device=device)
for seq,p in top3:
    print("How to"," ".join(seq),f"(P≈{p:.4f})")


[Dropout]  Epoch 1/5 — loss: 6.7994
[Dropout]  Epoch 2/5 — loss: 5.2874
[Dropout]  Epoch 3/5 — loss: 4.1878
[Dropout]  Epoch 4/5 — loss: 3.4492
[Dropout]  Epoch 5/5 — loss: 3.0396

Dropout-LSTM samples:
How to fix the health care disaster (P≈0.0211)
How to talk to your child doctor (P≈0.0133)
How to fix the health system actually (P≈0.0113)


### Bidrectional with Dropout

In [9]:
# Cell 3: Bidirectional + Dropout LSTM (fixed)


# 0) make sure pad_id is defined
pad_id  = vocab["[PAD]"]
pad_idx = pad_id

class BiDropoutLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, nlayers=2,
                 pad_idx=0, emb_drop=0.2, lstm_drop=0.3):
        super().__init__()
        self.hidden_dim = hid_dim
        self.embed      = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.emb_drop   = nn.Dropout(emb_drop)
        self.lstm       = nn.LSTM(
            emb_dim, hid_dim, nlayers,
            batch_first=True,
            bidirectional=True,
            dropout=lstm_drop
        )
        # now only projecting from the FORWARD half
        self.fc         = nn.Linear(hid_dim, vocab_size)

    def forward(self, x):
        emb = self.emb_drop(self.embed(x))  # (B, T, E)
        out, _ = self.lstm(emb)             # (B, T, 2*H)
        fwd = out[:, :, :self.hidden_dim]   # take only forward states → (B, T, H)
        return self.fc(fwd)                 # → (B, T, V)

# Instantiate
vocab_size = len(vocab) + 1      # +1 if your ids start at 1, with 0=PAD
embed_dim  = 128
hidden_dim = 128
num_layers = 1
model     = BiDropoutLSTM(vocab_size, embed_dim, hidden_dim, num_layers, pad_idx).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
n_epochs  = 5

# Training loop
for epoch in range(1, n_epochs+1):
    model.train(); total_loss = 0
    for xb, _ in loader:
        xb = xb.to(device)
        optimizer.zero_grad()
        logits = model(xb)                     # (B, T, V)
        B, T, V = logits.size()
        out = logits[:, :-1, :].reshape(-1, V) # shift for teacher forcing
        tgt = xb[:, 1:].reshape(-1)
        loss = criterion(out, tgt)
        loss.backward(); optimizer.step()
        total_loss += loss.item()
    print(f"[Bi+Dropout] Epoch {epoch}/{n_epochs} — loss: {total_loss/len(loader):.4f}")
    scheduler.step()

# Generate
print("\nBi-Dropout-LSTM samples (beam search):")
top3 = generate_beams("How to", model, tokenizer, id_to_token, pad_id,
                      max_ctx=5, beam_width=3, gen_len=5, device=device)
for seq, p in top3:
    print("How to", " ".join(seq), f"(P≈{p:.4f})")

print("\nBi-Dropout-LSTM samples (top-k sampling):")
for _ in range(3):
    out = generate_sampling(
        "How to", model, tokenizer, id_to_token, pad_id,
        temperature=0.8, top_k=30, device=device
    )
    print("How to", " ".join(out))


C:\Users\Putna\.conda\envs\en605645\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


[Bi+Dropout] Epoch 1/5 — loss: 7.0738
[Bi+Dropout] Epoch 2/5 — loss: 5.9215
[Bi+Dropout] Epoch 3/5 — loss: 5.2430
[Bi+Dropout] Epoch 4/5 — loss: 4.7730
[Bi+Dropout] Epoch 5/5 — loss: 4.4292

Bi-Dropout-LSTM samples (beam search):
How to be mindful while shoveling a (P≈0.0009)
How to be mindful while cleaning can (P≈0.0005)
How to be mindful while shoveling the (P≈0.0003)

Bi-Dropout-LSTM samples (top-k sampling):
How to prepare for your phone is
How to con the most terrible of
How to get the people deserve the


### Incorporating Keywords

In [10]:
### Cell 1: Bidirectional LSTM with keyword context
from torch.optim import AdamW

# make sure pad_id is defined from your vocab

pad_id = vocab["[PAD]"]
pad_idx = pad_id

# 0) Context builder using keywords + headline tokens
def make_context(head_ids, kw_ids, pad_id, max_ctx):
    seq = kw_ids + head_ids
    seq = seq[-max_ctx:]
    if len(seq) < max_ctx:
        seq = [pad_id] * (max_ctx - len(seq)) + seq
    return seq

# 1) Build dataset with sliding windows + keywords
pad_id  = vocab["[PAD]"]
max_ctx = 5
X2, y2 = [], []
for head_ids, kw_str in zip(tokenized, articles['keywords'].fillna('').tolist()):
    kw_ids = tokenizer.encode(kw_str.lower()).ids
    for i in range(1, len(head_ids)):
        window = head_ids[: i+1]
        ctx = make_context(window, kw_ids, pad_id, max_ctx)
        target = ctx[-1]
        if target == pad_id:
            continue
        X2.append(ctx[:-1])
        y2.append(target)

X_tensor = torch.tensor(X2, dtype=torch.long)
y_tensor = torch.tensor(y2, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)
loader  = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

# 0) Context builder using keywords + headline tokens
def make_context(head_ids, kw_ids, pad_id, max_ctx):
    seq = kw_ids + head_ids
    seq = seq[-max_ctx:]
    if len(seq) < max_ctx:
        seq = [pad_id] * (max_ctx - len(seq)) + seq
    return seq

# 1) Build dataset with sliding windows + keywords
pad_id  = vocab["[PAD]"]
max_ctx = 5
X2, y2 = [], []
for head_ids, kw_str in zip(tokenized, articles['keywords'].fillna('').tolist()):
    kw_ids = tokenizer.encode(kw_str.lower()).ids
    for i in range(1, len(head_ids)):
        window = head_ids[: i+1]
        ctx = make_context(window, kw_ids, pad_id, max_ctx)
        target = ctx[-1]
        if target == pad_id:
            continue
        X2.append(ctx[:-1])
        y2.append(target)

X_tensor = torch.tensor(X2, dtype=torch.long)
y_tensor = torch.tensor(y2, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)
loader  = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

# 2) Define the Bidirectional LSTM with Dropout + Forward-only projection
class BiHeadlineLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, nlayers=1,
                 pad_idx=0, emb_drop=0.3, lstm_drop=0.3):
        super().__init__()
        self.hidden_dim = hid_dim
        self.embed      = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.emb_drop   = nn.Dropout(emb_drop)
        self.lstm       = nn.LSTM(
            emb_dim, hid_dim, nlayers,
            batch_first=True,
            bidirectional=True,
            dropout=lstm_drop if nlayers>1 else 0.0
        )
        # project only forward hidden state
        self.fc         = nn.Linear(hid_dim, vocab_size)

    def forward(self, x):
        emb = self.emb_drop(self.embed(x))   # (B,T,E)
        out, _ = self.lstm(emb)              # (B,T,2H)
        # split: forward = out[..., :hidden_dim]
        fwd = out[:, :, :self.hidden_dim]    # (B,T,H)
        return self.fc(fwd)                  # (B,T,V)





def generate_beams(
    ctx_ids: str,
    model: nn.Module,
    tokenizer,
    id_to_token: dict,
    pad_id: int,
    max_ctx: int = 5,
    beam_width: int = 3,
    gen_len: int = 3,
    device: torch.device = torch.device("cpu")
    ):
    # 1) Tokenize & left-pad the initial context
    ids = tokenizer.encode(raw_ctx.lower()).ids
    if len(ids) < max_ctx:
        ctx = [pad_id] * (max_ctx - len(ids)) + ids
    else:
        ctx = ids[-max_ctx:]
    # beams: list of (generated_ids, log_prob)
    beams = [ (ctx.copy(), 0.0) ]
    
    model.eval()
    with torch.no_grad():
        for _ in range(gen_len):
            all_candidates = []
            for seq_ids, seq_logp in beams:
                # run the model on the current seq_ids
                x = torch.tensor([seq_ids], device=device)
                logits = model(x)                     # → (1, T, V)
                last_logits = logits[:, -1, :]        # → (1, V)
                logps = F.log_softmax(last_logits, dim=-1).squeeze(0)  # → (V,)

                # pick top beam_width next tokens
                top_logp, top_idx = logps.topk(beam_width)
                for logp, tok in zip(top_logp.tolist(), top_idx.tolist()):
                    new_seq = (seq_ids + [tok])[-max_ctx:]  # slide window
                    all_candidates.append((new_seq, seq_logp + logp))

            # keep only the top beam_width overall
            beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
    
    # Map to human‐readable
    results = []
    for seq_ids, logp in beams:
        # drop padding and the original context
        gen_part = seq_ids[-gen_len:]
        tokens   = [ id_to_token[i] for i in gen_part ]
        prob     = torch.exp(torch.tensor(logp))  # total sequence probability
        results.append((tokens, prob.item()))
    return results


# 3) Instantiate & train
vocab_size = len(vocab) + 1      # +1 if your ids start at 1, with 0=PAD
embed_dim  = 128
hidden_dim = 128
num_layers = 1
# 3) Instantiate & train with weight decay and gradient clipping


model     = BiHeadlineLSTM(vocab_size, embed_dim, hidden_dim, num_layers, pad_id).to(device)
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

for epoch in range(1, n_epochs+1):
    model.train(); total_loss=0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)                          # (B,T,V)
        B, T, V = logits.size()
        out    = logits[:, :-1, :].reshape(-1, V)
        tgt    = xb[:, 1:].reshape(-1)
        loss   = criterion(out, tgt)
        loss.backward()
        # clip gradients to stabilize training
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
    print(f"[Bi-LSTM with keywords] Epoch {epoch}/{n_epochs} — loss: {total_loss/len(loader):.4f}")
    scheduler.step()


ctx = make_context(tokenizer.encode("How to".lower()).ids,
                   tokenizer.encode("finance tips".lower()).ids,
                   pad_id, max_ctx=5)



# 4) Generate top-3
# 4) Generate top-3 (beam search) and sampling (as before)
print("\nBi-LSTM+keywords samples (beam search):")
top3 = generate_beams(
    "How to", model, tokenizer, id_to_token, pad_id,
    max_ctx=max_ctx, beam_width=3, gen_len=5, device=device
)
for seq, p in top3:
    print("How to", " ".join(seq), f"(P≈{p:.4f})")

print("\nBi-LSTM+keywords samples (sampling):")
for _ in range(3):
    seq = generate_sampling(
        "How to", model, tokenizer, id_to_token, pad_id,
        temperature=0.8, top_k=30, max_ctx=max_ctx, gen_len=5, device=device
        )
    print("How to", " ".join(seq))

[Bi-LSTM with keywords] Epoch 1/5 — loss: 6.6804
[Bi-LSTM with keywords] Epoch 2/5 — loss: 5.7184
[Bi-LSTM with keywords] Epoch 3/5 — loss: 5.2471
[Bi-LSTM with keywords] Epoch 4/5 — loss: 4.8942
[Bi-LSTM with keywords] Epoch 5/5 — loss: 4.6193

Bi-LSTM+keywords samples (beam search):
How to be mindful at the new (P≈0.0002)
How to be mindful while not so (P≈0.0002)
How to be mindful while not the (P≈0.0002)

Bi-LSTM+keywords samples (sampling):
How to work the best of the
How to be a key element prudence
How to be mindful at trump to


## Observations

Over the course of debugging, I discovered that the core problem was that the original LSTM was never really “seeing” the final real token in its hidden state. I first switched from right‐padding to left‐padding so that the last timestep fed to the LSTM corresponded to the final word (“to” in “how to”). When that alone didn’t suffice, I implemented a dynamic‐gather approach—computing the true sequence lengths and extracting the hidden state at the last non‐PAD index—ensuring the model’s final projection was conditioned on the actual last token. I then filtered out any windows whose target was PAD and restricted sliding windows to contexts of length ≥2 so that “how to” examples weren’t drowned out by one‐word or zero‐word prefixes. To further amplify the rare “how to” bigram (just 80 occurrences among 9,335 headlines), I replaced uniform sampling with a `WeightedRandomSampler` that inversely weighted context frequency—and even experimented with squaring those weights—to flood each training epoch with “how to” instances.

When these data‐level fixes still yielded generic continuations (“the”, “of”, “a”), I overhauled the training objective to a full sequence‐to‐sequence, teacher‐forcing loss: instead of learning only the next token per window, the model now predicts every subsequent token in the headline, multiplying the supervisory signal for each “how to” headline by its length. I added a `StepLR` scheduler to decay the learning rate on a timed schedule, tightened the tokenizer by expanding contractions and stripping possessives (so stray “s” tokens disappeared), and even experimented with more powerful architectures—bidirectional LSTMs, dropout regularization, and keyword‐augmented contexts—to give the model richer representations. Finally, recognizing that a deterministic beam search can loop on high‐probability words like “cancel drive the…,” I introduced sampling‐based generation: applying temperature scaling, top-k filtering, and no-repeat n-gram blocking to encourage diverse, meaningful continuations like **“do,” “get,”** or **“be”** after **“How to.”**

When the Bi-LSTM’s training loss crashes almost immediately to near zero, that’s a classic symptom of over-capacity and over-fitting: with twice as many parameters (forward + backward) as a unidirectional LSTM, it simply “memorizes” the training headlines—especially since we’re still forcing it with full teacher-forcing and haven’t constrained its backward pass at inference. Even though we sliced off the backward hidden states at output and added dropout, the backward LSTM still sees the entire future context during training and learns features you can’t actually use when generating left-to-right. Once it has enough capacity to perfectly reconstruct the next-token distribution, the loss “disappears” but the model’s generative behavior collapses to repeating memorized n-grams. Reducing embedding and hidden dimension, and number of layers helped.

For the keyword-augmented Bi-LSTM, I first enriched each training example by prepending its keyword IDs to the headline token window (via `make_context`), so the model learns to ground its predictions in both global topic cues and local headline context. I left-padded those combined sequences to a fixed length and used a full seq2seq (teacher-forcing) loss over every timestep, rather than just one next-token per window—multiplying the “how to…” signal by headline length.

On the modeling side, I switched to a bidirectional LSTM but explicitly projected **only** the forward hidden states into the output layer—eliminating the train/inference mismatch where backward states aren’t available at generation time. To prevent the two-fold parameter increase from instantly memorizing n-grams, I added dropout (0.3 on embeddings, 0.3 between LSTM layers), switched to AdamW with L2 weight decay (1e-4), and clipped gradients at norm 1.0. I also applied a StepLR schedule to halve the learning rate every 10 epochs. Finally, at generation time we support both beam‐search and temperature-scaled, top-k sampling (with optional no-repeat n-gram blocking) to produce diverse, coherent continuations like **“How to be mindful while enjoying…”** rather than loops of the same word.
